# Stable Pose Experiment Result Processing
After running the "Stable Pose" Experiments, we have got three files which preserve rotation matrice samples saved as numpy array format. They are:

```
.
└── pose_exp
    ├── topshell.npz                                   
    ├── mainshell.npz            
    └── insert_mold.npz
```

This notebook is aiming to compute the Euler angles of all pose candidates based on these rotation matrice samples.

In [29]:
import blenderproc as bproc
import argparse
import numpy as np
import bpy
import time
import math
from pathlib import Path

## 1. Load datas
* Rotation matrices shape-> [smaple_numbers x 3 x 3]

In [30]:
classes = ['mainshell', 'topshell', 'insert_mold']

R = []
builds = ["legacy", "UT1113-11901-7H", "UT12113-120BK-7H"]

for catogory in classes:
    loaded = np.load(f"./pose_exp/convexhull_UT1113/{catogory}.npz")
    R_temp = loaded['Rotation']
    R.append(R_temp)

## 2. Rotation to euler angles
Ref: https://learnopencv.com/rotation-matrix-to-euler-angles/

Same algorithm as MATLAB’s `rotm2euler()`

In [31]:

def isRotationMatrix(R):
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    
    return n<1e-6

def R2Euler(R):
    assert(isRotationMatrix(R))
    
    sy = math.sqrt(R[0,0]*R[0,0] + R[1,0]*R[1,0])
    singular = sy < 1e-6
    
    if not singular:
        pitch_x = math.atan2(R[2,1], R[2,2])
        roll_y  = math.atan2(-R[2,0], sy)
        yaw_z   = math.atan2(R[1,0], R[0,0])
    else:
        pitch_x = math.atan2(-R[1,2], R[1,1])
        roll_y = math.atan2(-R[2,0], sy)
        yaw_z = 0
    
    pitch_x = round(math.degrees(pitch_x) ,0)
    roll_y =round(math.degrees(roll_y) ,0)
    yaw_z = round(math.degrees(yaw_z) ,1) 

    if abs(pitch_x)==0:
        pitch_x = abs(pitch_x)
    roll_y = abs(roll_y)

    if abs(pitch_x)==180 and abs(roll_y)==0:
        pitch_x = abs(pitch_x)

    return np.array([pitch_x, roll_y, yaw_z])

np.set_printoptions(suppress=True)



## 3. Count all pose candidates

In [37]:
Euler_set = []
# From rotation to Euler
for Rot in R:
    poses = [] 
    for pose in Rot:
        poses.append(R2Euler(pose))
    Euler_set.append(poses)

# Count each candidate
Euler_dics = []
for Eulers in Euler_set:
    temp_dic = {}
    for Euler in Eulers:
        Euler_str = str(Euler[:2])
        if Euler[1] != 0.:
            # Due to fact that all parts are symmetric,
            # The Roll angle of stable poses should be always 0.
            temp_dic["Roll != 0"] = temp_dic.get("Roll != 0", 0) + 1
        elif 179<= abs(Euler[0]) and abs(Euler[0]) <= 179 and Euler[1]==0:
            temp_dic["[180.   0.]"] = temp_dic.get("[180.   0.]", 0) + 1
        else:
            temp_dic[Euler_str] = temp_dic.get(Euler_str, 0) + 1
    Euler_dics.append(temp_dic)


for i in range(len(Euler_dics)):
    # print('#'*35)
    print(f'\n {classes[i]} [Pithc, Roll] angles candidates')
    print('#'*45)
    samples_num = 0
    for pose,num in Euler_dics[i].items():
        samples_num += num
        print(pose, '\t num =', num)
    print(f"Total Smaples \t num =  {samples_num}")


 mainshell [Pithc, Roll] angles candidates
#############################################
[180.   0.] 	 num = 3126
[-12.   0.] 	 num = 1627
[-90.   0.] 	 num = 164
[90.  0.] 	 num = 76
Roll != 0 	 num = 6
[89.  0.] 	 num = 1
Total Smaples 	 num =  5000

 topshell [Pithc, Roll] angles candidates
#############################################
[180.   0.] 	 num = 4003
[-90.   0.] 	 num = 24
[0. 0.] 	 num = 935
[90.  0.] 	 num = 29
Roll != 0 	 num = 9
Total Smaples 	 num =  5000

 insert_mold [Pithc, Roll] angles candidates
#############################################
[180.   0.] 	 num = 2958
[-25.   0.] 	 num = 1956
[31.  0.] 	 num = 18
[90.  0.] 	 num = 16
Roll != 0 	 num = 38
[-178.    0.] 	 num = 3
[145.   0.] 	 num = 1
[-177.    0.] 	 num = 5
[-24.   0.] 	 num = 1
[-176.    0.] 	 num = 4
Total Smaples 	 num =  5000
